In [6]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
from pathlib import Path
import hvplot
import hvplot.pandas
from IPython.display import Markdown
from sklearn.model_selection import train_test_split
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler


In [7]:
# Set the file path
filepath = Path("../machine-learning-project/MSFT_df.csv")

# Read the CSV located at the file path into a Pandas DataFrame
df = pd.read_csv(filepath, parse_dates=True, infer_datetime_format=True)

# Print the DataFrame
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Returns,50 Day MA,Upper,Lower,Trading Signal,Entry/Exit
0,2011-02-10,27.930000,27.940001,27.290001,27.500000,21.864422,76672400,-0.016804,27.9144,28.636182,26.778219,1.0,1.0
1,2011-02-11,27.760000,27.809999,27.070000,27.250000,21.665649,83939700,-0.009091,27.9386,28.268699,26.919901,1.0,0.0
2,2011-02-14,27.209999,27.270000,26.950001,27.230000,21.649748,56766200,-0.000734,27.9454,28.224377,26.951022,1.0,0.0
3,2011-02-15,27.040001,27.330000,26.950001,26.959999,21.561775,44116500,-0.009916,27.9442,27.959069,26.945130,1.0,0.0
4,2011-02-16,27.049999,27.070000,26.600000,27.020000,21.609766,70817900,0.002226,27.9478,28.004014,26.963787,1.0,0.0


In [8]:
# Visualize exit position relative to close price
exit = df[df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = df[df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = df[['50 Day MA']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I          :Curve   [index]   (Close)
   .Curve.II         :Curve   [index]   (50 Day MA)
   .Scatter.Close.I  :Scatter   [index]   (Close)
   .Scatter.Close.II :Scatter   [index]   (Close)

In [9]:
# Set initial capital
initial_capital = float(100000)

# Set the share size
share_size = initial_capital / df['Close']

# Portfolio Holding shares
df['Portfolio Holdings'] = share_size

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
df['Portfolio Cash'] = (df['Portfolio Holdings']* df['Close']) - initial_capital

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
df['Portfolio Total'] = df['Portfolio Cash'] + (df['Portfolio Holdings']*  df['Close'])

# Calculate the portfolio daily returns
df['Portfolio Daily Returns'] = df['Portfolio Total'].pct_change()

# Calculate the cumulative returns
df['Portfolio Cumulative Returns'] = (1 + df['Portfolio Daily Returns']).cumprod() - 1


# Print the DataFrame
df.head(50)

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Returns,50 Day MA,Upper,Lower,Trading Signal,Entry/Exit,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
0,2011-02-10,27.930000,27.940001,27.290001,27.500000,21.864422,76672400,-0.016804,27.9144,28.636182,26.778219,1.0,1.0,3636.363636,0.0,100000.0,NaN,NaN
1,2011-02-11,27.760000,27.809999,27.070000,27.250000,21.665649,83939700,-0.009091,27.9386,28.268699,26.919901,1.0,0.0,3669.724771,0.0,100000.0,0.0,0.0
2,2011-02-14,27.209999,27.270000,26.950001,27.230000,21.649748,56766200,-0.000734,27.9454,28.224377,26.951022,1.0,0.0,3672.420187,0.0,100000.0,0.0,0.0
3,2011-02-15,27.040001,27.330000,26.950001,26.959999,21.561775,44116500,-0.009916,27.9442,27.959069,26.945130,1.0,0.0,3709.198939,0.0,100000.0,0.0,0.0
4,2011-02-16,27.049999,27.070000,26.600000,27.020000,21.609766,70817900,0.002226,27.9478,28.004014,26.963787,1.0,0.0,3700.962187,0.0,100000.0,0.0,0.0
5,2011-02-17,26.969999,27.370001,26.910000,27.209999,21.761717,57207300,0.007032,27.9546,28.167967,26.996632,1.0,0.0,3675.119565,0.0,100000.0,0.0,0.0
6,2011-02-18,27.129999,27.209999,26.990000,27.059999,21.641756,68667800,-0.005513,27.9512,28.029600,26.981599,1.0,0.0,3695.491573,0.0,100000.0,0.0,0.0
7,2011-02-22,26.780001,27.100000,26.520000,26.590000,21.265869,60889000,-0.017369,27.9414,27.604411,26.926989,1.0,0.0,3760.812314,0.0,100000.0,0.0,0.0
8,2011-02-23,26.530001,26.860001,26.430000,26.590000,21.265869,60234100,0.000000,27.9264,27.661294,26.855106,1.0,0.0,3760.812314,0.0,100000.0,0.0,0.0
9,2011-02-24,26.639999,27.059999,26.500000,26.770000,21.409822,64494200,0.006769,27.9168,27.874133,26.812667,1.0,0.0,3735.524777,0.0,100000.0,0.0,0.0


In [10]:
# Visualize exit position relative to total portfolio value
exit = df[df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = df[df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize total portoflio value for the investment
total_portfolio_value = df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I                    :Curve   [index]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [index]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [index]   (Portfolio Total)

In [11]:
# Prepare DataFrame for metrics
metrics = [
    'Annual Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio']

columns = ['Backtest']

# Initialize the DataFrame with index set to evaluation metrics and column as `Backtest` (just like PyFolio)
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)
portfolio_evaluation_df

,Backtest
Annual Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN


In [12]:
# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'] = df['Portfolio Cumulative Returns']

# Calculate annualized return
portfolio_evaluation_df.loc['Annual Return'] = (
    df['Portfolio Daily Returns'].mean() * 252
)

# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'] = (
    df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

# Calculate Sharpe Ratio
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    df['Portfolio Daily Returns'].mean() * 252) / (
    df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

# Calculate Downside Return
sortino_ratio_df = df[['Portfolio Daily Returns']].copy()
sortino_ratio_df.loc[:,'Downside Returns'] = 0

target = 0
mask = sortino_ratio_df['Portfolio Daily Returns'] < target
sortino_ratio_df.loc[mask, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2
portfolio_evaluation_df

portfolio_evaluation_df.head()

,Backtest
Annual Return,-1.96115e-15
Cumulative Returns,NaN
Annual Volatility,1.98545e-15
Sharpe Ratio,-0.987764


In [30]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss


In [36]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = abs(row['Portfolio Holdings'])* entry_share_price
        share_size = row['Portfolio Holdings']
        entry_share_price = row['Close']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['Close'] * share_size)
        exit_share_price = row['Close']
        profit_loss =  exit_portfolio_holding - entry_portfolio_holding  
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'MSFT',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df.tail(50)
trade_evaluation_df.to_csv('trade_evaluation_df.csv', index = True)